## ETL - Visualización y Storytelling
### Documentación del Proyecto Final del Curso

Dada la naturaleza de la historia que se quiere contar y siguiendo con los lineamientos básicos de modelado de datos, se propone trabajar con la siguiente definición con el fin de soportar la presentación de la información:


* Fecha

|Campo   | Descripción  |
|:---:|:-------------:|
| FechaPK  | Identificador del fecha a granualaridad de día. La forma de armarlo es concatenando año,mes y día (sin comas ni separadores)  |
| Año  | Año de la fecha del contexto  |
| Mes  | Mes de la fecha del contexto  |
| NombreMes  | Nombre del mes de la fecha del contexto  |
| Semestre  |  Semestre de la fecha del contexto  |
| Trimestre  | Trimestre de la fecha del contexto   |
| Fecha  | Fecha del contexto  |

* FactCovid

|Campo   | Descripción  |
|:---:|:-------------:|
| Covid_Id  | Llave primaria generada artificialmente  |
|  FechaFK | Llave foránea a la tabla Fecha  |
| Departamento  |  Departamento de Colombia del cual se produjo el resultado de casos de Covid para una fecha dada |
| Cantidad Casos  |  Número de ocurrencias. Medida. |
|  Estado |  Puede ser Leve, Moderado, Grave o Fallecido. Algunas filas no tienen valor desde el organismo Estatal que las provee |

* FactBancos

|Campo   | Descripción  |
|:---:|:-------------:|
| bancos_Id  | Llave primaria generada artificialmente  |
|  FechaFK | Llave foránea a la tabla Fecha  |
| Banco  |  Nombre de la entidad Bancaria |
| Altura  | Indica la altura de mora en meses en que se encuentra el saldo adeudado: Bruto= Saldo Bruto que le adeudan los clientes al banco. Vigente: Saldo en 0 días de mora , xx meses: saldo en mora en xx meses.  |
| Total Comercial  | Saldo total adeudado por los clientes al banco de la Cartera Comercial (Empresas) |
| Total Consumo | Saldo total adeudado por los clientes al banco de la Cartera Consumo (Tarjetas Crédito) |
| Total Microcredito  | Saldo total adeudado por los clientes al banco de la Cartera Mircrocredito (Pymes) |
| Total Vivienda |  Saldo total adeudado por los clientes al banco de la Cartera Vivienda (Hipotecas) |

* FactSeguros

|Campo   | Descripción  |
|:---:|:-------------:|
| bancos_Id  | Llave primaria generada artificialmente  |
|  FechaFK | Llave foránea a la tabla Fecha  |
|  RAMO |  Modalidades de seguro en las que se divide el conjunto de seguros ofrecidos por una compañía aseguradora según el criterio de agrupar riesgos |
| SEGMENTO_RAMO  |  Agrupación  global para distinguir entre los seguros de vida y los seguros comerciales |
| COMPANIA_ORIGINA  | Nombres de la compañía de seguro sin homologar, abiertos por segmento ramo  |
| COMPANIA | Nombres de las compañías de seguros homologados y unicos  |
| M_SINIESTROS_INCURRIDOS  |  Valor pagado por las compañías al beneficiario de una poliza por concepto de siniestros ocurridos |
| M_PRIMAS_DEVENGADAS  | Prima real recibida luego de los descuentos y/o devoluciones que hayan a lugar por finalización anticipada del contrato  |
| M_PRIMAS_EMITIDAS | Valor Recibido (precio del seguro) pagado por los clientes  |
| M_RESULTADO_TECNICO |  Refleja el resultado obtenido por el negocio exclusivamente de seguros, sin considerar los gastos de administración y productos financieros que se derivan de la gestión |
| M_GASTOS_ADMINISTRATIVOS_Y_DE | Gastos administrativos en los que ha incurrido la aseguradora para el ejercicio de su labor  |




![title](MER_V&S.drawio.png)

De la imagen anterior tenemos que este modelo cuenta con tres tablas de hechos: (FactCovid, FactBancos, FactSeguros) y una única dimensión compartida (Fecha). Se tiene claro que algunos atributos podrían ir perfectamente en su propia dimensión, pero dado que solo servirían para filtrar una única tabla de hechos y a que se busca manejar un modelo simple, se ha decidido mantenerlos en las tablas de hechos.

Empezamos cargando las librerias necesarias y los dataframes iniciales que serán la representación inicial de los archivos encontrados en bases de datos públicas. Tambien se tienen dos carpetas asociadas al proyecto:
* SourceFiles: Donde se deben ubicar los archivos planos obtenidos de fuentes públicas. Serán los datos de entrada del programa.
* DWHFiles: Donde la ETL guardará los csv finales a ser usados en el dashboard. Si ya existen archivos anteriormente, el programa los reemplaza.

La recomendación general es que si se produce alguna actualización en alguno de los archivos fuente, se ejecute la totalidad de la presente ETL para mayor facilidad de uso.

In [1]:
import numpy as np
import pandas as pd

PATH = "SourceFiles/"
DESTINATION_PATH='DWHFiles/'

def carga_covid():
    df_covid = pd.read_csv(PATH+'Casos_positivos_de_COVID-19_en_Colombia.csv')
    return df_covid

def carga_bancos():
    df_bancos = pd.read_excel(PATH+'Bench_bancos.xlsx')
    return df_bancos

def carga_seguros():
    df_seguros = pd.read_excel(PATH+'AMP_BENCH_FASECOLDA_TOTAL.xlsx')
    return df_seguros

df_covid   = carga_covid()
df_bancos  = carga_bancos()
df_seguros = carga_seguros()

Se procede a realizar una limpieza de los dataframes, ya que algunas de las columnas de los archivos base no son necesarias:

In [2]:
df_bancos.drop(df_bancos.columns[[0, 3, 10]], axis = 1, inplace = True)
df_seguros = df_seguros[["FECHA", "RAMO", "SEGMENTO_RAMO", "COMPANIA_ORIGINA", "COMPANIA", "M_SINIESTROS_INCURRIDOS", "M_PRIMAS_DEVENGADAS", "M_PRIMAS_EMITIDAS", "M_RESULTADO_TECNICO", "M_GASTOS_ADMINISTRATIVOS_Y_DE"]]
df_seguros.rename(columns={'FECHA': 'FECHAFK'}, inplace=True)

Requerimos que todas las tablas tengan como llave de tiempo un número, de la misma forma que df_seguros tiene su campo "FECHA". Por lo que usamos las columnas existentes en df_covid y df_bancos para calcular dicha nueva columna. Empezaremos con df_bancos:

In [3]:
df_bancos.columns = df_bancos.columns.str.strip()

def monthToNum(MesEsp):
    return {
            'Enero': '01',
            'Febrero':'02',
            'Marzo': '03',
            'Abril': '04',
            'Mayo': '05',
            'Junio': '06',
            'Julio': '07',
            'Agosto': '08',
            'Septiembre': '09', 
            'Octubre': '10',
            'Noviembre': '11',
            'Diciembre': '12'
    }[MesEsp]

def calculoFecha(row):
    return str(row['Ano'])+monthToNum(row['Mes'])+'01'


df_bancos['FECHAFK'] = df_bancos.apply (lambda row: calculoFecha(row), axis=1)
#Se remueven columnas innecesarias ahora para df_bancos
df_bancos.drop(df_bancos.columns[[6, 7]], axis = 1, inplace = True)

Continuamos con el df_covid, que ya tiene un campo de fecha y solamente se debe ajustar el formato. Tambien aprovechamos y ajustamos un poco los nombres de las columnas restantes del dataframe:

In [4]:
df_covid["fecha reporte web"] = pd.to_datetime(df_covid["fecha reporte web"]).dt.strftime("%Y%m%d")
df_covid.rename(columns={'Nombre departamento': 'Departamento' , 'Unidad de medida de edad': 'Cantidad Casos', 'fecha reporte web': 'FECHAFK'}, inplace=True)

Se procede a generar la dimensión Fecha, la cual contendrá todas las fechas desde el año 2000 hasta el año 2024. Esto con el fin de contar con un margen amplio de fechas para ser elegidas por los usuarios en el dashboard/historia:

In [5]:
def create_date_table(start='2000-01-01', end='2024-12-31'):
    start_ts = pd.to_datetime(start).date()

    end_ts = pd.to_datetime(end).date()

    #record timetsamp is empty for now
    dates =  pd.DataFrame(columns=['Record_timestamp'],
        index=pd.date_range(start_ts, end_ts))
    dates.index.name = 'Fecha'

    days_names = {
        i: name
        for i, name
        in enumerate(['Lunes', 'Martes', 'Miercoles',
                      'Jueves', 'Viernes', 'Sabado', 
                      'Domingo'])
    }
    
    month_names = {
        i: name
        for i, name
        in enumerate(['Enero', 'Febrero', 'Marzo',
                      'Abril', 'Mayo', 'Junio', 
                      'Julio', 'Agosto', 'Septiembre',
                     'Octubre', 'Noviembre', 'Diciembre'])
    }

    dates['Dia'] = dates.index.dayofweek.map(days_names.get)
    dates['Semana'] = dates.index.isocalendar().week #dates.index.week
    dates['Mes'] = dates.index.month
    dates['NombreMes'] = (dates.index.month-1).map(month_names.get)
    dates['Cuarter'] = dates.index.quarter
    dates['Semestre'] = dates.index.month.map(lambda mth: 1 if mth <7 else 2)
    dates['Anio'] = dates.index.year
    dates['FechaPK'] = dates.index.strftime("%Y%m%d")
    dates.reset_index(inplace=True)
    dates.index.name = 'date_id'
    return dates

df_fecha = create_date_table()

Se valida la integridad de las columnas necesarias para el dashboard:

In [6]:
print(df_covid)
print(df_bancos)
print(df_seguros)
print(df_fecha)

       Departamento Cantidad Casos   FECHAFK     Estado
0           CORDOBA            152  20220203       Leve
1             SUCRE            134  20200714       Leve
2          CASANARE             54  20201101       Leve
3            NARIÑO             34  20210106  Fallecido
4      CUNDINAMARCA              6  20210413        NaN
...             ...            ...       ...        ...
48452         SUCRE             29  20201216       Leve
48453         VALLE              2  20200606        NaN
48454      CASANARE             54  20210414       Leve
48455       QUINDIO              2  20210408  Fallecido
48456       QUINDIO              1  20200614       Leve

[48457 rows x 4 columns]
                      Banco      Altura  TOTAL COMERCIAL  TOTAL CONSUMO  \
0      ARCO GRUPO BANCOLDEX   +12 MESES         20552.41           0.00   
1      ARCO GRUPO BANCOLDEX   +18 MESES             0.00           0.00   
2      ARCO GRUPO BANCOLDEX    +4 MESES             0.00           0.00   
3 

Finalmente, se guardan los archivos finales en csv que serán el modelo de datos a ser cargados en la herramienta de visualización elegida:

In [7]:
df_covid.index.name = 'covid_id'
df_covid.to_csv(DESTINATION_PATH+'Covid.csv',encoding="UTF-8")
df_bancos.index.name = 'bancos_id'
df_bancos.to_csv(DESTINATION_PATH+'Bancos.csv',encoding="UTF-8")
df_seguros.index.name = 'seguros_id'
df_seguros.to_csv(DESTINATION_PATH+'Seguros.csv',encoding="UTF-8")
df_fecha.to_csv(DESTINATION_PATH+'Fecha.csv',encoding="UTF-8")

Si se desean realizar actualizaciones sobre los datasources ya configurados en Google, se debe contar con el acceso a los mismos (cualquier persona tendrá acceso si cuenta con los siguientes enlaces). La configuración final es:

* Bancos: https://datastudio.google.com/datasources/4c17ffd9-97f4-42a6-a00b-c915bd81da25
* Seguros: https://datastudio.google.com/datasources/0e07a957-328a-459e-986d-087b96c39f1d
* Covid: https://datastudio.google.com/datasources/737e951f-0a00-4bea-afbd-cddc5998fbd9
* Fecha: https://datastudio.google.com/datasources/e1ba9110-cde3-46d2-ae28-b0102270c962

Se pueden subir varios archivos de datos para futuras actualizaciones que tengan la misma estructura definida o es posible eliminar la base del archivo actual y subir una nueva que, por ejemplo, tenga columnas nuevas. Si se agregan columnas, esto no afecta la definición actual de los tableros ni es necesario dar permisos adicionales, solamente se debe seguir el wizard de Google con el fin de eliminar el archivo anterior y subir el nuevo. Si se piensa eliminar columnas, se debe validar con el encargado de las visualizaciones para medir el impacto de dicho cambio en la definición del dashboard actual. No se recomienda realizar.